<a href="https://colab.research.google.com/github/SourCherries/FR-APSDEHAL/blob/master/FR_APSDEHAL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Face recognition CNN by APSDEHAL from

Minimal script to run forward pass on model using initial parameters. Uncertain whether initial paramters are random. Goal is to pinpoint source of error when running main.py where weights appear to be on cpu despite putting entire model onto gpu.

Based on [pytorch example](https://pytorch.org/hub/pytorch_vision_vgg/) and on [CNN by APSDEHAL](https://github.com/apsdehal/Face-Recognition).

In [1]:
!git clone https://github.com/SourCherries/FR-APSDEHAL.git

Cloning into 'FR-APSDEHAL'...
remote: Enumerating objects: 89, done.
remote: Counting objects: 100% (25/25), done.
remote: Compressing objects: 100% (25/25), done.
remote: Total 89 (delta 12), reused 0 (delta 0), pack-reused 64
Unpacking objects: 100% (89/89), done.


In [2]:
import sys
sys.path.insert(1, '/content/FR-APSDEHAL')

import torch
from PIL import Image
from torchvision import transforms
from model_noargs import Network
from constants import IMG_SIZE, NUM_CHANNELS

from torchvision.io import read_image

# Load image(s)

In [3]:
judi = read_image("/content/FR-APSDEHAL/judi-dench-256.png")
kathy = read_image("/content/FR-APSDEHAL/kathleen-turner-256.png")
input_batch = torch.cat([torch.unsqueeze(judi, 0), torch.unsqueeze(kathy, 0)], dim=0)

print(input_batch.shape)
print(input_batch.dtype)

torch.Size([2, 3, 256, 256])
torch.uint8


In [4]:
from PIL import Image
input_image = Image.open("/content/FR-APSDEHAL/judi-dench-256.png")
preprocess = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])
input_tensor = preprocess(input_image)
input_batch = input_tensor.unsqueeze(0) # create a mini-batch as expected by the model

print(input_batch.shape)
print(input_batch.dtype)
print(input_batch.min())
print(input_batch.max())

torch.Size([1, 3, 256, 256])
torch.float32
tensor(-2.1179)
tensor(2.4286)


# Test model

In [5]:
model = Network()
with torch.no_grad():
    output = model(input_batch)

/content/FR-APSDEHAL/model_noargs.py:57: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return nn.functional.log_softmax(self.fully_connected2(x))


In [6]:
print(output.shape)
print(output.dtype)
print(output)

torch.Size([1, 15])
torch.float32
tensor([[-2.7510, -2.7007, -2.7054, -2.6757, -2.6779, -2.7398, -2.7487, -2.7057,
         -2.7231, -2.7803, -2.7318, -2.6871, -2.6362, -2.6597, -2.7079]])


In [7]:
probabilities = torch.nn.functional.softmax(output[0], dim=0)
print(probabilities)

tensor([0.0639, 0.0672, 0.0668, 0.0689, 0.0687, 0.0646, 0.0640, 0.0668, 0.0657,
        0.0620, 0.0651, 0.0681, 0.0716, 0.0700, 0.0667])
